<a href="https://colab.research.google.com/github/AnthonyAugust/Projetos-Finalizados/blob/main/Projeto_de_cria%C3%A7%C3%A3o_de_uma_base_de_dados_e_treinamento_da_rede_YOLO_(Subconjunto_de_Imagens_Menor_Dataset_Classifica_errado).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Preparando o ambiente

!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17564, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 17564 (delta 35), reused 7 (delta 7), pack-reused 17507 (from 3)
Receiving objects: 100% (17564/17564), 16.64 MiB | 22.99 MiB/s, done.
Resolving deltas: 100% (12029/12029), done.
/content/yolov5/yolov5


In [7]:
# Baixando o subconjunto do COCO

import os
from pycocotools.coco import COCO
import requests
from tqdm import tqdm

# Pastas
os.makedirs('coco/train/images', exist_ok=True)
os.makedirs('coco/train/labels', exist_ok=True)
os.makedirs('coco/val/images', exist_ok=True)
os.makedirs('coco/val/labels', exist_ok=True)

# Baixar anotações COCO
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip

# Classes de interesse
CLASSES = ["dog", "bicycle", "truck"]
NUM_TRAIN = 500
NUM_VAL = 100

# Função para baixar imagens e labels
def baixar_subset(coco_json, split, num_imgs):
    coco = COCO(coco_json)
    cat_ids = coco.getCatIds(catNms=CLASSES)
    img_ids = coco.getImgIds(catIds=cat_ids)
    img_ids = img_ids[:num_imgs]

    for img_id in tqdm(img_ids):
        img_info = coco.loadImgs(img_id)[0]
        img_url = img_info['coco_url']
        img_path = f'coco/{split}/images/{img_info["file_name"]}'

        # Baixar imagem
        if not os.path.exists(img_path):
            img_data = requests.get(img_url).content
            with open(img_path, 'wb') as f:
                f.write(img_data)

        # Criar labels YOLO
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_ids)
        anns = coco.loadAnns(ann_ids)
        label_path = f'coco/{split}/labels/{img_info["file_name"].replace(".jpg",".txt")}'
        with open(label_path, 'w') as f:
            for ann in anns:
                x,y,w,h = ann['bbox']
                x_c = (x + w/2)/img_info['width']
                y_c = (y + h/2)/img_info['height']
                w /= img_info['width']
                h /= img_info['height']
                cls = cat_ids.index(ann['category_id'])
                f.write(f"{cls} {x_c} {y_c} {w} {h}\n")

# Baixar subconjunto para treino e validação
baixar_subset('annotations/instances_train2017.json', 'train', NUM_TRAIN)
baixar_subset('annotations/instances_val2017.json', 'val', NUM_VAL)


--2025-08-26 10:09:55--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.201.217, 3.5.3.14, 3.5.20.54, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.201.217|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  96.5MB/s    in 2.5s    

2025-08-26 10:09:57 (96.5 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

loading annotations into memory...
Done (t=15.65s)
creating index...
index created!


100%|██████████| 12/12 [00:02<00:00,  4.80it/s]


loading annotations into memory...
Done (t=0.58s)
creating index...
index created!


100%|██████████| 2/2 [00:00<00:00,  5.02it/s]


In [8]:
# Criando arquivo de configuração

# Criar arquivo data.yaml para YOLOv5
yaml_content = f"""
train: ../coco/train
val: ../coco/val

nc: {len(CLASSES)}
names: {CLASSES}
"""
with open("coco.yaml", "w") as f:
    f.write(yaml_content)

In [9]:
# Treinar a YOLOv5 com parâmetros Leves
!python train.py --img 416 --batch 8 --epochs 50 --data coco.yaml --weights yolov5s.pt --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-08-26 10:10:45.408569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756203045.435973    3516 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756203045.443629    3516 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756203045.464525    3516 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756203045.464589    3516 computation_placer.cc:177] computation placer already registere

In [10]:
# Testar Modelo
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.25 --source coco/val/images

detect: weights=['runs/train/exp/weights/best.pt'], source=coco/val/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-430-g459d8bf0 Python-3.12.11 torch-2.8.0+cu126 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/2 /content/yolov5/yolov5/coco/val/images/000000007386.jpg: 288x416 (no detections), 214.6ms
image 2/2 /content/yolov5/yolov5/coco/val/images/000000424162.jpg: 352x416 (no detections), 261.1ms
Speed: 1.0ms pre-process, 237.9ms inference, 0.4ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp
